In [1]:
import os
import textwrap
from typing import cast

import langextract as lx
from langextract.data import AnnotatedDocument

In [2]:
prompt = textwrap.dedent("""
Ekstrak infromasi entitas nama, lokasi, waktu, dan organisasi sesuai urutan kemunculannya pada teks. Gunakan teks sesuai diberikan, jangan melakukan parafrase.
""")

examples = [
    lx.data.ExampleData(
        text="Setelah kemerdekaan Indonesia dideklarasikan pada tanggal 17 Agustus 1945, Badan Pendidikan Tinggi Republik Indonesia (BPTRI) didirikan pada tanggal 19 Agustus 1945 di Jakarta. BPTRI memiliki dua fakultas (waktu itu disebut perguruan tinggi), yaitu Perguruan Tinggi Kedokteran dan Perguruan Tinggi Hukum/Kesusasteraan. Ketua BPTRI adalah Prof. dr. Sarwono Prawirohardjo.",
        extractions=[
            lx.data.Extraction(extraction_class="lokasi", extraction_text="Indonesia"),
            lx.data.Extraction(extraction_class="waktu", extraction_text="17 Agustus 1945"),
            lx.data.Extraction(
                extraction_class="organisasi", extraction_text="Badan Pendidikan Tinggi Republik Indonesia"
            ),
            lx.data.Extraction(extraction_class="organisasi", extraction_text="BPTRI"),
            lx.data.Extraction(extraction_class="waktu", extraction_text="19 Agustus 1945"),
            lx.data.Extraction(extraction_class="lokasi", extraction_text="Jakarta"),
            lx.data.Extraction(extraction_class="organisasi", extraction_text="Perguruan Tinggi Kedokteran"),
            lx.data.Extraction(extraction_class="organisasi", extraction_text="Perguruan Tinggi Hukum/Kesusasteraan"),
            lx.data.Extraction(extraction_class="nama", extraction_text="Prof. dr. Sarwono Prawirohardjo"),
        ],
    ),
    lx.data.ExampleData(
        text="Universitas Indonesia (UI) secara resmi memulai kegiatannya pada 2 Februari 1950 dengan presiden (saat ini disebut rektor) pertamanya Ir. R.M. Pandji Soerachman Tjokroadisoerio. Kantor Presiden Universiteit Indonesia mula-mula berkedudukan di Jakarta, tepatnya di gedung Fakultas Kedokteran di Jl. Salemba Raya no. 6, kemudian dipindahkan ke salah satu bangunan bekas pabrik madat atau opium di Jl. Salemba Raya no. 4, Jakarta. Tanggal 2 Februari 1950 kemudian dijadikan hari kelahiran Universitas Indonesia.",
        extractions=[
            lx.data.Extraction(extraction_class="organisasi", extraction_text="Universitas Indonesia"),
            lx.data.Extraction(extraction_class="organisasi", extraction_text="UI"),
            lx.data.Extraction(extraction_class="waktu", extraction_text="2 Februari 1950"),
            lx.data.Extraction(extraction_class="nama", extraction_text="Ir. R.M. Pandji Soerachman Tjokroadisoerio"),
            lx.data.Extraction(extraction_class="lokasi", extraction_text="Kantor Presiden Universiteit Indonesia"),
            lx.data.Extraction(extraction_class="lokasi", extraction_text="Jakarta"),
            lx.data.Extraction(extraction_class="lokasi", extraction_text="Fakultas Kedokteran"),
            lx.data.Extraction(extraction_class="lokasi", extraction_text="Jl. Salemba Raya no. 6"),
            lx.data.Extraction(extraction_class="lokasi", extraction_text="Jl. Salemba Raya no. 4, Jakarta"),
        ],
    ),
]

In [3]:
input_text = textwrap.dedent("""
Universitas Indonesia (disingkat UI) adalah perguruan tinggi di Indonesia. Kampus utamanya terletak di bagian Utara dari Depok, Jawa Barat tepat di perbatasan antara Depok dan wilayah Jakarta Selatan, sementara kampus utama lainnya terdapat di Salemba, Jakarta Pusat. UI merupakan institusi pendidikan tinggi tertua di Indonesia. Beberapa perguruan tinggi negeri terkemuka di Indonesia seperti Institut Teknologi Bandung, Universitas Airlangga, Institut Pertanian Bogor, Universitas Hasanuddin, Universitas Negeri Jakarta, dan Politeknik Negeri Jakarta pada awalnya merupakan bagian dari UI hingga kemudian memisahkan diri menjadi institusi tersendiri. UI juga merupakan salah satu dari tiga perguruan tinggi terbaik di Indonesia bersama dengan Universitas Gadjah Mada dan Institut Teknologi Bandung.
""")

input_text_2 = textwrap.dedent("""
Cikal-bakal terbentuknya Universitas Indonesia adalah ketika pemerintah kolonial Belanda mendirikan sebuah sekolah yang bertujuan untuk menghasilkan asisten dokter tambahan yang memegang sertifikat untuk melakukan perawatan-perawatan tingkat dasar serta mendapatkan gelar Dokter Jawa (Javanese doctor). Secara resmi, UI memulai kegiatannya pada 2 Februari 1950 dengan presiden (saat ini disebut rektor) pertamanya Ir. R.M. Pandji Soerachman Tjokroadisoerio. Tanggal tersebut dijadikan hari kelahiran Universitas Indonesia.
""")

input_text_3 = textwrap.dedent("""
Berangsur-angsur fakultas-fakultas yang berada di daerah memisahkan diri membentuk lembaga pendidikan yang berdiri sendiri. Pada tanggal 2 Maret 1959 Fakultas Teknik dan Fakultas Ilmu Pengetahuan Alam di Bandung memisahkan diri menjadi Institut Teknologi Bandung. Selanjutnya pada 1 September 1963 Fakultas Pertanian dan Fakultas Kedokteran Hewan UI memisahkan diri pula menjadi Institut Pertanian Bogor (IPB), fakultas di Surabaya menjadi Universitas Airlangga, dan di Makassar menjadi Universitas Hasanuddin. Pada 1964 Fakultas Keguruan dan Ilmu Pendidikan menjadi Institut Keguruan dan Ilmu Pendidikan (IKIP) Jakarta dan kini berubah kembali menjadi Universitas Negeri Jakarta. 
""")

documents = [
    lx.data.Document(text=input_text),
    lx.data.Document(text=input_text_2),
    lx.data.Document(text=input_text_3),
]

In [4]:
config_llamacpp = lx.factory.ModelConfig(
    model_id="hf:MaziyarPanahi/Mistral-7B-Instruct-v0.3-GGUF:*Q4_K_M.gguf",
    provider="LlamaCppLanguageModel",
    provider_kwargs=dict(
        n_gpu_layers=-1,
        n_ctx=4096,
        verbose=False,
        completion_kwargs=dict(
            temperature=0,
            seed=42,
        ),
    ),
)

config_openrouter = lx.factory.ModelConfig(
    model_id="meta-llama/llama-3.3-70b-instruct:free",
    provider="OpenAILanguageModel",  # explicitly set the provider to `OpenAILanguageModel`
    provider_kwargs=dict(
        base_url="https://openrouter.ai/api/v1",
        api_key=os.environ["OPENROUTER_API_KEY"],
    ),
)

model_llamacpp = lx.factory.create_model(config_llamacpp)
model_openrouter = lx.factory.create_model(config_openrouter)

llama_context: n_ctx_per_seq (4096) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64 

In [5]:
# # pass by document

# results = lx.extract(
#     model=model_openrouter,
#     text_or_documents=documents,
#     prompt_description=prompt,
#     examples=examples,
#     extraction_passes=2,  # multi-extraction pass; better result
#     fence_output=False,
#     use_schema_constraints=False,
#     debug=False,
#     max_workers=4,
# )


# # casting for type hint
# results = cast(Iterable[AnnotatedDocument], results)
# result_list: list[AnnotatedDocument] = list(results)

In [6]:
# pass by text

results = [
    lx.extract(
        model=model_llamacpp,
        text_or_documents=text,
        prompt_description=prompt,
        examples=examples,
        extraction_passes=3,  # multi-extraction pass; better result
        fence_output=False,
        use_schema_constraints=False,
        debug=False,
        max_workers=4,
    )
    for text in [input_text, input_text_2, input_text_3]
]

result_list = cast(list[AnnotatedDocument], results)

LangExtract: model=hf:MaziyarPanahi/Mistral-7B-Instruct-v0.3-GGUF:*Q4_K_M.gguf, current=800 chars, processed=0 chars:  [00:55]
LangExtract: model=hf:MaziyarPanahi/Mistral-7B-Instruct-v0.3-GGUF:*Q4_K_M.gguf, current=522 chars, processed=0 chars:  [00:32]
LangExtract: model=hf:MaziyarPanahi/Mistral-7B-Instruct-v0.3-GGUF:*Q4_K_M.gguf, current=680 chars, processed=0 chars:  [00:58]


In [7]:
for extracted in result_list[2].extractions:  # ty:ignore[not-iterable]
    print(f"{extracted.extraction_text=}")
    print(f"{extracted.extraction_class=}")
    print(f"{extracted.attributes=}")
    print(f"{extracted.char_interval=}")
    print()

extracted.extraction_text='2 Maret 1959'
extracted.extraction_class='waktu'
extracted.attributes={}
extracted.char_interval=CharInterval(start_pos=138, end_pos=150)

extracted.extraction_text='Bandung'
extracted.extraction_class='lokasi'
extracted.attributes={}
extracted.char_interval=CharInterval(start_pos=205, end_pos=212)

extracted.extraction_text='Fakultas Teknik'
extracted.extraction_class='organisasi'
extracted.attributes={}
extracted.char_interval=CharInterval(start_pos=151, end_pos=166)

extracted.extraction_text='Fakultas Ilmu Pengetahuan Alam'
extracted.extraction_class='organisasi'
extracted.attributes={}
extracted.char_interval=CharInterval(start_pos=171, end_pos=201)

extracted.extraction_text='Institut Teknologi Bandung'
extracted.extraction_class='organisasi'
extracted.attributes={}
extracted.char_interval=CharInterval(start_pos=237, end_pos=263)

extracted.extraction_text='1 September 1963'
extracted.extraction_class='waktu'
extracted.attributes={}
extracted.char_inter

In [8]:
lx.io.save_annotated_documents(result_list, output_name="extractions.jsonl", output_dir=".")

for idx, result in enumerate(result_list):
    html_content = lx.visualize(result)
    with open(f"extraction_{idx}.html", "w") as f:
        if hasattr(html_content, "data"):
            f.write(html_content.data)  # For Jupyter/Colab
        else:
            f.write(html_content)

LangExtract: Saving to extractions.jsonl: 3 docs [00:00, 1830.51 docs/s]

✓ Saved 3 documents to extractions.jsonl
